# Extract EICs from LC-MS data and plot
Written by Kelly Styles, 12th November 2024

## Preparing your MS1 data
You will need to convert your existing Agilent `.D` files to the `mzXML` or `mzML` formats. 

It has been tested on the old Agilent LC-MS data and it works quite well. To do this, you will need to:
1. Convert your Agilent '.D' files to MassHunter format using the 'LC-SQ ChemStation Translator' software provided by Agilent.
2. Convert this data to either `mz` formats using [Proteowizard - msConvert](https://proteowizard.sourceforge.io/download.html)

<div style="border-left: 4px solid #0074D9; padding: 10px; background-color: #E8F4FD;">
<b>Note:</b> 
    This Notebook will work for MS1 data from both MS1 and MS2 datafiles. It has been tested using Agilent data.
</div>

## Usage
Up the top of Jupyter Notebook/Lab, click `Run > Run All Cells` and a new window should pop up in your browser. Upload your `mzXML` files in the order you would like them to be plotted, then input your selected parameters and play around with image output options. Both SVG and JPG image files will be generated for each plot and can be foung in the same folder as this Notebook.


## Troubleshooting
Several issues could cause an `Error` message. First check if using labels that the filenames in the table match the filename in the uploaded files section, sometimes extra spaces might appear if these are copied and pasted. If using LaTeX formatting syntax for the label names, check this is also correct. Also, if using named colours, check these are spelled correctly. If none of these are relevant, there will likely be some error messages at the bottom of this Notebook. Copy and paste this and pass it on to me and I can try to see what's going on.

In [ ]:
# Checks installation status of packages and install if absent

import sys
import subprocess

def install_and_import(packages):
    for package in packages:
        try:
            __import__(package)
            print(f"'{package}' is already installed.")
        except ImportError:
            print(f"'{package}' not found. Installing...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"'{package}' has been installed successfully.")

# List of packages you want to check and install if necessary
packages = ["seaborn", "pyopenms==3.2.0", "gradio==5.1.0", "numpy", "matplotlib", "pandas"]

# Run the function
install_and_import(packages)


In [ ]:
import os
from pyopenms import MSExperiment, MzXMLFile, MzMLFile
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter
import gradio as gr

# Function to load mzXML data
def load_mz(file_path):
    experiment = MSExperiment()
    if file_path.lower().endswith(".mzxml"):
        MzXMLFile().load(file_path, experiment)
    elif file_path.lower().endswith(".mzml"):
        MzMLFile().load(file_path, experiment)
    else:
        print(f"{file_path} is not in the correct format. It must be either in `mzML` or `mzXML` file format")
    return experiment

# Baseline correction (simple method: subtracting the median intensity)
def baseline_correction(intensities):
    baseline = np.median(intensities)
    corrected = np.array(intensities) - baseline
    corrected[corrected < 0] = 0  # No negative intensities
    return corrected

# Extract EIC based on target mass and tolerance
def extract_eic(experiment, target_mass, tolerance=0.5):
    eic_times, eic_intensities = [], []
    for spectrum in experiment:
        if spectrum.getMSLevel() == 1:  # MS1 spectra
            mzs, intensities = spectrum.get_peaks()
            mask = (mzs >= float(target_mass) - float(tolerance)) & (mzs <= float(target_mass) + float(tolerance))
            if np.any(mask):  # if any points fall within the mass range
                eic_times.append(spectrum.getRT())
                eic_intensities.append(np.sum(intensities[mask]))
    return np.array(eic_times), baseline_correction(np.array(eic_intensities))

def extract_tic(experiment):
    tic_times, tic_intensities = [], []
    for spectrum in experiment:
        if spectrum.getMSLevel() == 1:  # MS1 spectra only
            _, intensities = spectrum.get_peaks()
            tic_times.append(spectrum.getRT())  # Retention time
            tic_intensities.append(np.sum(intensities))  # Sum all intensities for TIC
    return np.array(tic_times), baseline_correction(np.array(tic_intensities))

def process_mz_files_eic(file_paths, target_masses, tolerance=0.5):
    all_eic_data = []
    
    for file_path in file_paths:
        try:
            experiment = load_mz(file_path)
            eic_data = []
            for mass in target_masses:
                eic_times, eic_intensities = extract_eic(experiment, mass, tolerance)
                eic_data.append((eic_times, eic_intensities))
            all_eic_data.append((file_path, eic_data))
        except RuntimeError as e:
            print(f"Error loading file {file_path}: {e}")
            return None
    
    return all_eic_data

def process_mz_files_tic(file_paths):
    all_tic_data = []
    
    for file_path in file_paths:
        try:
            exp = load_mz(file_path)
            all_tic_data.append((file_path, exp))
        except RuntimeError as e:
            print(f"Error loading file {file_path}: {e}")
            return None
    
    return all_tic_data

def plot_ms_data_gradio_eic(data, labels_dict, mass_colour_dict, image_title, offset, yaxis_on, xaxis_on, axes_off, fill_under, alpha, dpi, window_length, polyorder):
    # Create subplots
    sns.set_style("ticks")
    fig, axes = plt.subplots(nrows=len(data), ncols=1, sharey=True, figsize=(10, len(data)))
    plt.subplots_adjust(wspace=0.2, hspace=0.2)
    
    # Initialize max_y to keep track of the highest y-value (intensity)
    max_y = float('-inf')
    for file_path, eic_data in data:
        for times, intensities in eic_data:
            max_y = max(max_y, max(intensities))  # Update max_y with the highest intensity found
    
    print("Maximum y-value (intensity) across all data:", max_y)
    print(mass_colour_dict)
    # Iterate through each file's data
    for i, (file_path, eic_data) in enumerate(data):  # eic_data corresponds to each file's extracted data
        print(i, file_path)
        bump = max_y * 0.1
        target_masses = list(mass_colour_dict.keys())
        base_offset = len(target_masses) * bump
        
        #for mass, (times, intensities) in zip(target_masses, eic_data):
        for j, (mass, (times, intensities)) in enumerate(list(zip(target_masses, eic_data))):
            times_in_minutes = [t / 60 for t in times]
            smoothed_intensities = savgol_filter(intensities, window_length, polyorder)
            colour = mass_colour_dict.get(float(mass), 'black')
            
            if offset == True:
                total_offset = base_offset
            else:
                total_offset = 0 
            
            # Assign a higher zorder for the more recent lines and fills
            line_zorder = j + 1
            fill_zorder = j
        
            # Plot EIC with or without legend based on subplot index
            if i == 0:
                sns.lineplot(x=times_in_minutes, y=smoothed_intensities + total_offset, ax=axes[i], label=f"$\it{{m/z}}$ {mass}", color=colour, zorder=line_zorder)
            else:
                sns.lineplot(x=times_in_minutes, y=smoothed_intensities + total_offset, ax=axes[i], color=colour, zorder=line_zorder)
            
            if fill_under:
                axes[i].fill_between(times_in_minutes, total_offset, smoothed_intensities + total_offset, color=colour, alpha=float(alpha), zorder=fill_zorder)  
        
            base_offset -= bump # plot 
        
        axes[i].set_xlim(left=0, right=max(times_in_minutes))    
        
        if labels_dict:
            try:
                label = labels_dict[os.path.basename(file_path)]
            except KeyError:
                continue
        else:
            label = os.path.basename(file_path)
            
        axes[i].text(max(times_in_minutes) * 1.01, max_y * 0.3, label)  # Label each subplot with the file name
        axes[i].yaxis.get_offset_text().set_x(-0.04)
        
        if i < len(data) - 1:
            axes[i].set_xticklabels([])  # Hide tick labels
        
        # Adjust y-axis display
        for i, ax in enumerate(axes):
            if i < len(axes) - 1:
                if not yaxis_on and not xaxis_on:
                    ax.tick_params(left=False, bottom=False)
                    ax.yaxis.set_visible(False) 
                    sns.despine(ax=ax, top=True, right=True, left=True, bottom=True)
                elif not yaxis_on and xaxis_on:
                    sns.despine(ax=ax, top=True, right=True, left=True, bottom=False, offset=5)
                    ax.tick_params(left=False)
                    ax.yaxis.set_visible(False) 
                elif yaxis_on and not xaxis_on:
                    sns.despine(ax=ax, top=True, right=True, left=False, bottom=True, offset=5)
                    ax.tick_params(bottom=False)
                else:
                    sns.despine(ax=ax, top=True, right=True, left=False, bottom=False, offset=5)
            elif axes_off:
                sns.despine(ax=ax, top=True, right=True, left=True, bottom=False, offset=5)
            else:
                sns.despine(ax=ax, top=True, right=True, left=False, bottom=False, offset=5)
    
    legend = axes[0].legend(frameon=False, loc="center right", bbox_to_anchor=(1.17,1.5))
    legend.set_title("Legend:", prop={'weight': 'bold'})
    axes[-1].set_xlabel("Retention time (mins)")
    axes[-1].set_ylabel("Counts")
    svg = image_title + ".svg"
    jpg = image_title + ".jpg"
    print(f"Saving image to {image_title}")
    plt.savefig(svg, format="svg", bbox_inches='tight')
    plt.savefig(jpg, format="jpg", bbox_inches='tight', dpi=int(dpi))

    return(jpg)

def plot_ms_data_gradio_tic(data, labels_dict, image_title, yaxis_on, xaxis_on, axes_off, fill_under, alpha, dpi, window_length, polyorder):
    # Create subplots
    sns.set_style("ticks")
    fig, axes = plt.subplots(nrows=len(data), ncols=1, sharey=True, figsize=(10, len(data)))
    plt.subplots_adjust(wspace=0.2, hspace=0.2)
    
    # Find maximum y-value across all TIC data for consistent scaling
    max_y = float('-inf')
    for file_path, eic_data in data:
        # Summing intensities to get the TIC for each time point
        tic = eic_data.calculateTIC()
        retention_times, intensities = tic.get_peaks()
        max_y = max(max_y, max(intensities))
    
    print("Maximum y-value (intensity) across all TIC data:", max_y)
    
    # Iterate through each file's data
    for i, (file_path, exp_data) in enumerate(data):
        tic = exp_data.calculateTIC()
        times, intensities = tic.get_peaks()
        times_in_minutes = [t / 60 for t in times]
        # Smooth the TIC if specified
        smoothed_tic = savgol_filter(intensities, window_length, polyorder)
        
        # Plot TIC with or without legend based on subplot index
        sns.lineplot(x=times_in_minutes, y=smoothed_tic, ax=axes[i], label="TIC", color="black", zorder=2, legend=False)
        
        # Fill under the curve if specified
        if fill_under:
            axes[i].fill_between(times_in_minutes, 0, smoothed_tic, color="black", alpha=float(alpha), zorder=1)
        
        # Set x-axis limits
        axes[i].set_xlim(left=0, right=max(times_in_minutes))
        
        # Label each subplot with the file name
        if labels_dict:
            try:
                label = labels_dict[os.path.basename(file_path)]
            except KeyError:
                continue
        else:
            label = os.path.basename(file_path)

        axes[i].text(max(times_in_minutes) * 1.01, max_y * 0.3, label)  # Label each subplot with the file name
        axes[i].yaxis.get_offset_text().set_x(-0.04)
        
        if i < len(data) - 1:
            axes[i].set_xticklabels([])  # Hide tick labels
        
        # Adjust y-axis display
        for i, ax in enumerate(axes):
            if i < len(axes) - 1:
                if not yaxis_on and not xaxis_on:
                    ax.tick_params(left=False, bottom=False)
                    ax.yaxis.set_visible(False) 
                    sns.despine(ax=ax, top=True, right=True, left=True, bottom=True)
                elif not yaxis_on and xaxis_on:
                    sns.despine(ax=ax, top=True, right=True, left=True, bottom=False, offset=5)
                    ax.tick_params(left=False)
                    ax.yaxis.set_visible(False) 
                elif yaxis_on and not xaxis_on:
                    sns.despine(ax=ax, top=True, right=True, left=False, bottom=True, offset=5)
                    ax.tick_params(bottom=False)
                else:
                    sns.despine(ax=ax, top=True, right=True, left=False, bottom=False, offset=5)
            elif axes_off:
                sns.despine(ax=ax, top=True, right=True, left=True, bottom=False, offset=5)
            else:
                sns.despine(ax=ax, top=True, right=True, left=False, bottom=False, offset=5)
    
    legend = axes[0].legend(frameon=False, loc="center right", bbox_to_anchor=(1.17,1.5))
    legend.set_title("Legend:", prop={'weight': 'bold'})
    axes[-1].set_xlabel("Retention time (mins)")
    axes[-1].set_ylabel("Counts")
    svg = image_title + ".svg"
    jpg = image_title + ".jpg"
    print(f"Saving image to {image_title}")
    plt.savefig(svg, format="svg", bbox_inches='tight')
    plt.savefig(jpg, format="jpg", bbox_inches='tight', dpi=int(dpi))

    return(jpg)

def process(file_paths, labels_df, masses_df, tolerance, image_title, offset, y_axis_on, x_axis_on, axes_off, fill_under, alpha, dpi, window_length, polyorder, extract_tic):
    
    mz_files = [file for file in file_paths if file.lower().endswith((".mzxml", ".mzml"))]   
    
    mass_colour_dict = {
            float(row['Mass']): row['Colour']
            for _, row in masses_df.iterrows()
            if row['Mass'] and row['Colour']
        }
    print(mass_colour_dict)
    labels_dict = {
           row['File']: row['Label']
            for _, row in labels_df.iterrows()
            if row['File'] and row['Label']
        }
    
    if extract_tic:
        data = process_mz_files_tic(mz_files)
        fig = plot_ms_data_gradio_tic(data, labels_dict, image_title, y_axis_on, x_axis_on, axes_off, fill_under, alpha, dpi, window_length, polyorder)
    else:
        data = process_mz_files_eic(mz_files, mass_colour_dict.keys(), tolerance)
        fig = plot_ms_data_gradio_eic(data, labels_dict, mass_colour_dict, image_title, offset, y_axis_on, x_axis_on, axes_off, fill_under, alpha, dpi, window_length, polyorder)
    
    return fig  # Return the plot for display in Gradio


In [ ]:
with gr.Blocks(theme=gr.themes.Citrus()) as demo:
    gr.Markdown("# Plot EICs\nUse the following inputs to customize your plot.")
    gr.Markdown("*v1.0 - Written by Kelly Styles*")
    # Input Files Section
    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            gr.Markdown("### Input Files:\nUpload your mzXML or mzML files for analysis. You can use LaTeX formatting between two `$` \
                to make text italics or bold\n e.g., **$\it{** *text* **}$** or **$\\bf{ text }$**")
            files_input = gr.Files(label="Upload mzXML Files")
            labels_df = gr.DataFrame(label="Label names", headers=["File", "Label"], row_count=12)
        with gr.Column(scale=1, min_width=300):
            gr.Markdown("### Target masses and colours:\nProvide mass and color pairs for highlighting specific masses. \
                Ensure you use named MatPlotLib colours as listed [here](https://matplotlib.org/stable/gallery/color/named_colors.html) \
                or an HTML colour HEX code.")
            tic_input = gr.Checkbox(label="Plot TIC (overrides any inputted masses below)")
            tolerance = gr.Textbox(label="EIC tolerance", value=0.5)
            masses_input = gr.DataFrame(label="Masses", headers=["Mass", "Colour"], row_count=6)

    # Plot Options Section
    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            gr.Markdown("### Plot Options:\nSet the plot title and choose additional display options.")
            image_title_input = gr.Textbox(label="Output image basename", value="eic_image")
            dpi=gr.Textbox(label="dpi for JPG file", value=400)
            offset_input = gr.Checkbox(label="Offset EICs")
            y_axis_on_input = gr.Checkbox(label="Show Y Axis")
            x_axis_on_input = gr.Checkbox(label="Show X Axis")
            axes_toggle = gr.Checkbox(label="Turn off all axes")
            fill_under_input = gr.Checkbox(label="Fill area under line")
            alpha_input = gr.Textbox(label="Fill alpha transparency", value=0.6)
        with gr.Column(scale=1, min_width=300):
            gr.Markdown("### (Advanced) Smoothing Parameters:\nCustomize the Savitzky-Golay smoothing filter settings.")
            window_length_input = gr.Number(label="Window Length", value=11)
            polyorder_input = gr.Number(label="Polyorder", value=2)

    with gr.Row():
        submit_button = gr.Button("Generate Plot")
    with gr.Row():
        #plot_output = gr.Plot(label="Generated Plot", elem_id="plot_output")
        plot_output = gr.Image(label="Generated Plot", elem_id="plot_output")
        
    submit_button.click(
        fn=process, 
        inputs=[
            files_input,
            labels_df,
            masses_input,
            tolerance,
            image_title_input,
            offset_input,
            y_axis_on_input,
            x_axis_on_input,
            axes_toggle,
            fill_under_input,
            alpha_input,
            dpi,
            window_length_input,
            polyorder_input,
            tic_input
        ], 
        outputs=plot_output
    )
    


demo.launch(inbrowser=True)